## Parameter pair correlation

This notebook allows comparison of user-specified pairs of parameters from the algorithm,
supporting examination of parameter correlation.

In [ ]:
import arviz as az
import numpy as np
from emutools.utils import load_param_info
from inputs.constants import RUN_IDS, RUNS_PATH, PRIMARY_ANALYSIS, BURN_IN
from emutools.calibration import plot_3d_param_hist

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / 'output/calib_full_out.nc')
idata = idata.sel(draw=np.s_[BURN_IN:])

Available parameters for comparison are:

In [ ]:
list(idata.posterior.keys())

In [ ]:
plot_3d_param_hist(idata, 'natural_immunity_period', 'ifr_adjuster', load_param_info()['abbreviations'])